# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [80]:
with open("real_estate_sales_data.txt" ,'r', encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [81]:
from langchain.text_splitter import CharacterTextSplitter

In [82]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 200,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [83]:
docs = text_splitter.create_documents([real_estate_sales])

In [84]:
docs[0]

Document(page_content='[客户问题] 新款手机与旧款有什么区别?\n[销售回答] 新款手机在处理器、相机以及电池续航方面都有显著提升，可以让您享受更流畅的操作体验，更美的照片以及更长的使用时间。\n\n\\d+\\.\n[客户问题] 这款手机耐用吗?\n[销售回答] 这款手机采用了最新的耐摔耐刮屏幕技术，同时还提供了一年的保修服务，确保您的使用无忧。')

In [85]:
len(docs)

16

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [86]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [87]:
query = "手机好不好"

In [88]:
answer_list = db.similarity_search(query)

In [89]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 我不常换手机，这款手机能用几年?
[销售回答] 我们的手机设计寿命至少为五年，而且我们会提供长期的系统更新和技术支持。

\d+\.
[客户问题] 我需要更好的摄像功能。
[销售回答] 这款手机配备了先进的多摄像头系统，不仅主摄像头分辨率高，还加入了超广角和长焦镜头，满足您不同场景的拍摄需求。

[客户问题] 我老公喜欢看球赛，屏幕显示效果好不好?
[销售回答] 我们手机的屏幕采用了高清显示技术，色彩鲜艳，对比度高，非常适合观看体育赛事和电影。

\d+\.
[客户问题] 我总是担心手机掉水里，这款机有防水功能吗?
[销售回答] 是的，我们的这款手机具备IP68级的防水防尘功能，可以承受一定时间和深度的浸水，让您日常使用无忧。

[客户问题] 这手机能连接我的智能手表和耳机吗？
[销售回答] 当然可以，我们的手机支持最新的蓝牙技术，可以与大多数智能设备无缝连接。

\d+\.
[客户问题] 我在找能做工作演示的手机，这款手机显示大文档怎么样？
[销售回答] 这款手机不仅屏幕大而清晰，而且有专门优化的办公软件，大文档的显示和编辑都非常流畅。

[客户问题] 如果手机有问题怎么办?
[销售回答] 如果遇到任何问题，您可以随时联系我们的客服中心，我们将提供专业的技术支持，并在必要时为您提供维修服务。

\d+\.
[客户问题] 我是学生，买不起太贵的手机。 
[销售回答] 我们有提供多款性价比很高的手机，它们的功能全面，性能稳定，是学生朋友的理想选择。



In [90]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [91]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [92]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A0B97CCFA0>, search_kwargs={'k': 3})

In [93]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我不常换手机，这款手机能用几年?
[销售回答] 我们的手机设计寿命至少为五年，而且我们会提供长期的系统更新和技术支持。

\d+\.
[客户问题] 我需要更好的摄像功能。
[销售回答] 这款手机配备了先进的多摄像头系统，不仅主摄像头分辨率高，还加入了超广角和长焦镜头，满足您不同场景的拍摄需求。

[客户问题] 我老公喜欢看球赛，屏幕显示效果好不好?
[销售回答] 我们手机的屏幕采用了高清显示技术，色彩鲜艳，对比度高，非常适合观看体育赛事和电影。

\d+\.
[客户问题] 我总是担心手机掉水里，这款机有防水功能吗?
[销售回答] 是的，我们的这款手机具备IP68级的防水防尘功能，可以承受一定时间和深度的浸水，让您日常使用无忧。

[客户问题] 这手机能连接我的智能手表和耳机吗？
[销售回答] 当然可以，我们的手机支持最新的蓝牙技术，可以与大多数智能设备无缝连接。

\d+\.
[客户问题] 我在找能做工作演示的手机，这款手机显示大文档怎么样？
[销售回答] 这款手机不仅屏幕大而清晰，而且有专门优化的办公软件，大文档的显示和编辑都非常流畅。



In [98]:
docs = topK_retriever.get_relevant_documents("手机存储空间大吗？")

In [99]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这款手机支持快充吗?
[销售回答] 当然，我们的手机支持最新的快充技术，只需半小时就能充满50%的电量。

\d+\.
[客户问题] 存储空间够用吗?
[销售回答] 这款手机提供了多种存储选项，最高支持512GB的存储空间，确保您有足够的空间来存储视频、照片和应用。

[客户问题] 我的工作需要经常出差，电池续航怎么样?
[销售回答] 这款手机配备了大容量电池，经过优化，常规使用可以持续超过一天，而且我们还提供移动电源配件，出差也不怕手机没电。

\d+\.
[客户问题] 我对摄影有些要求，这款手机的拍照功能行吗?
[销售回答] 这款手机的相机由专业团队调校，提供专业模式和丰富的拍摄场景选择，满足您对高质量摄影的追求。

[客户问题] 我老公喜欢看球赛，屏幕显示效果好不好?
[销售回答] 我们手机的屏幕采用了高清显示技术，色彩鲜艳，对比度高，非常适合观看体育赛事和电影。

\d+\.
[客户问题] 我总是担心手机掉水里，这款机有防水功能吗?
[销售回答] 是的，我们的这款手机具备IP68级的防水防尘功能，可以承受一定时间和深度的浸水，让您日常使用无忧。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [100]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [101]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\ProgramData\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


### 提取向量数据库中的`销售回答`

In [102]:
docs = retriever.get_relevant_documents(query)

D:\ProgramData\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [103]:
docs[0].page_content

IndexError: list index out of range

In [104]:
docs[0].page_content.split("[销售回答] ")

IndexError: list index out of range

In [105]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

IndexError: list index out of range

In [24]:
ans

'这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

#### 尝试各种问题

In [107]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [108]:
query = "我想能连接手表的手机"

print(sales(query))

['这款手机不仅屏幕大而清晰，而且有专门优化的办公软件，大文档的显示和编辑都非常流畅。']


In [109]:
print(sales(query, 0.75))

['这款手机不仅屏幕大而清晰，而且有专门优化的办公软件，大文档的显示和编辑都非常流畅。']


In [110]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\ProgramData\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



D:\ProgramData\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['我们有提供多款性价比很高的手机，它们的功能全面，性能稳定，是学生朋友的理想选择。', '当然有，我们提供了多种分期付款方案，您可以根据自己的财务状况选择最适合的方案，减轻一次性支付的压力。', '这款手机提供了多种存储选项，最高支持512GB的存储空间，确保您有足够的空间来存储视频、照片和应用。', '这款手机配备了先进的多摄像头系统，不仅主摄像头分辨率高，还加入了超广角和长焦镜头，满足您不同场景的拍摄需求。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [29]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [31]:
qa_chain({"query": "你们小区有200万的房子吗？"})

{'query': '你们小区有200万的房子吗？',
 'result': '对不起，我无法回答这个问题，因为我是一个人工智能，没有实时的房地产信息。建议你直接联系房地产经纪人或者查阅相关房地产网站获取信息。'}

In [32]:
qa_chain({"query": "小区吵不吵"})

{'query': '小区吵不吵',
 'result': '这个小区特别注重居住体验，有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。所以，小区应该不会太吵。'}

In [33]:
print(sales("小区吵不吵"))

['这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']


## 加载 FAISS 向量数据库已有结果

In [34]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings())

In [35]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [36]:
qa_chain({"query": "我想买别墅，你们有么"})

{'query': '我想买别墅，你们有么',
 'result': '对不起，我无法提供这样的服务。我是一个人工智能，我可以帮助回答问题，但我不能出售商品或房产。'}

In [37]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [38]:
qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助你购买别墅，因为我是一个人工智能，没有提供房地产服务的功能。'}

In [39]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [40]:
result = qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [41]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮你购买别墅。我是一个AI助手，我主要用来提供信息和回答问题。',
 'source_documents': []}